In [34]:
import os
from openai import OpenAI
from google.colab import userdata

# Set up your OpenAI API key
# Make sure to add your API key to Colab secrets under the name 'OPENAI_API_KEY'
# For more info, see https://colab.research.google.com/notebooks/integrations.ipynb
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Initialize the OpenAI client
# This client is used to interact with the OpenAI API
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# 1. Types of Memory

1.  **Contextual (Short-Term) Memory:** Keeps conversations coherent in a single session.
2.  **Long-Term (Persistent) Memory:** Remembers facts and preferences across sessions.
3.  **Semantic Memory:** The model’s built-in factual and conceptual knowledge.
4.  **Working Memory:** Handles multi-step reasoning and problem-solving within a single query.
5.  **External / Tool-Augmented Memory (RAG):** Extends the model’s capacity by retrieving info from external sources.

## 1. Contextual (Short-Term) Memory

Keeps conversations coherent in a single session by remembering recent interactions. This type of memory is crucial for maintaining flow and understanding within a limited timeframe.

In [35]:
# Step 1: First message to set context
# This message introduces the user and sets the initial context for the conversation.
response1 = client.chat.completions.create(
    model="gpt-4o-mini",  # Small, fast model for demo
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "My name is Alex."}
    ]
)
print(response1.choices[0].message.content)

Hello, Alex! How can I assist you today?


In [36]:
# Step 2: Second message reuses context (conversation continues)
# By including the previous messages, the model remembers the user's name from the first interaction.
response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        # Include the previous messages to maintain context
        {"role": "assistant", "content": response1.choices[0].message.content},
        {"role": "user", "content": "What is my name?"}
    ]
)
print(response2.choices[0].message.content)

You mentioned your name is Alex. How can I help you today, Alex?


## 2. Long-Term (Persistent) Memory

Remembers facts and preferences across sessions, allowing the model to maintain consistency and personalization over extended periods.

In [7]:
# Persistent file for memory
MEMORY_FILE = "memory.txt"

def save_memory(data):
    with open(MEMORY_FILE, "w") as f:
        f.write(data)

def load_memory():
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, "r") as f:
            return f.read().strip()
    return None

# Step 1: User gives info
user_message = "My name is Alex"

# Step 2: Ask GPT-4 if this should be remembered
decision = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "system", "content": "Decide if the user's message contains important facts to store such as name, preferences and so on"},
        {"role": "user", "content": user_message}
    ]
)

print("Model decision:", decision.choices[0].message.content)

# Step 3: If model says it's important, trigger 'store_memory'
if "remember" in decision.choices[0].message.content.lower():
    save_memory(user_message)
    print("Stored in memory:", user_message)

# Step 4: Later, retrieve memory and use it in a new conversation
stored_info = load_memory()
if stored_info:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a friendly assistant."},
            {"role": "user", "content": f"Previously you learned: {stored_info}. Now greet me using that info."}
        ]
    )
    print(response.choices[0].message.content)


Model decision: Storing this information: User's name is Alex.


In [47]:
"""
Simple LLM Long Memory System
- Stores user facts in JSON file
- Uses OpenAI function calling to save/recall info
- Perfect for Google Colab experiments
"""

import json
import os
from openai import OpenAI

MEMORY_FILE = "memory.json"  # Where memories are stored

def save_memory(key: str, value: str):
    """Store a key-value pair in memory file"""
    print("Memory updating...")
    memory = {}
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, "r") as f:
            memory = json.load(f)

    memory[key] = value
    with open(MEMORY_FILE, "w") as f:
        json.dump(memory, f, indent=2)
    return "Stored successfully"

def load_memory(query: str):
    """Search for stored memories matching the query"""
    print("Reading memory...")
    if not os.path.exists(MEMORY_FILE):
        return "No memories stored"

    with open(MEMORY_FILE, "r") as f:
        memory = json.load(f)

    if not memory:
        return "No memories stored"

    # Search for relevant info
    # query_lower = query.lower()
    # relevant = {k: v for k, v in memory.items()
    #            if query_lower in k.lower() or query_lower in str(v).lower()}

    # if relevant:
    #     return json.dumps(relevant)
    # else:
    #     return f"No memory found for: {query}"
    return memory # Return the entire memory dictionary

# Function tools for the AI to call

# Tool definitions
tools = [
    {
        "type": "function",
        "function": {
            "name": "store_fact",
            "description": "Store user info like name, age, etc.",
            "parameters": {
                "type": "object",
                "properties": {
                    "key": {"type": "string"},
                    "value": {"type": "string"}
                },
                "required": ["key", "value"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "recall_fact",
            "description": "Get stored user info",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            }
        }
    }
]

def chat_with_memory(user_input: str):
    """Main function that handles conversation with memory capabilities"""
    client = OpenAI()

    messages = [
        {"role": "system", "content": "You have memory tools. When user shares personal info, store it. When they ask about themselves, recall it first."},
        {"role": "user", "content": user_input}
    ]

    # First API call - may trigger tool usage
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    # Handle tool calls if any
    if response.choices[0].message.tool_calls:
        messages.append(response.choices[0].message)

        for tool_call in response.choices[0].message.tool_calls:
            args = json.loads(tool_call.function.arguments)

            # Execute the appropriate tool
            if tool_call.function.name == "store_fact":
                result = save_memory(args["key"], args["value"])
            elif tool_call.function.name == "recall_fact":
                result = load_memory(args["query"])

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result) # Ensure the content is a string
            })

        # Second API call with tool results
        final_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )


        return final_response.choices[0].message.content

    return response.choices[0].message.content

# ==================== DEMO ====================



user_text = input("\n💬 You: ")

print("\n🤖 Assistant:")
response = chat_with_memory(user_text)
print(response)


💬 You: how old Am i

🤖 Assistant:
Reading memory...
You are 54 years old, Jo.


## 3. Semantic Memory

The model’s built-in factual and conceptual knowledge, enabling it to understand and generate human-like text based on its training data.

In [48]:
# Define the input message for the model
input_message = "Write a short scientific article about nature." # Using the original prompt content

# Create a response request using the responses endpoint
response = client.responses.create(
    model="gpt-4o-mini",  # Using gpt-4o-mini
    input=input_message,
    max_output_tokens=100
)

# Extract and print the model's output text
print(response.output_text)

**The Importance of Biodiversity in Ecosystem Health**

**Abstract:**  
Biodiversity, the variety of life forms within a given ecosystem, is crucial for maintaining ecological balance and resilience. This article explores the importance of biodiversity, the threats it faces, and the impact on ecosystem health and human well-being.

**Introduction:**  
Nature is a complex web of interconnected organisms, functioning collectively to sustain life on Earth. Biodiversity includes not only the variety of species but also genetic diversity within those species and


# 2. Embeddings

## 1. Vectors

In [53]:
import numpy as np

def generate_vector(n):
    """Generate a vector with n dimensions filled with random values"""
    return np.random.rand(n)

# Example usage
number = 110
vector = generate_vector(number)

print(f"Input: {number}")
print(f"Vector: {vector}")
print(f"Shape: {vector.shape}")


Input: 110
Vector: [0.38671283 0.41435042 0.11562326 0.68164822 0.82984494 0.34270913
 0.95964889 0.47047841 0.55288474 0.1822974  0.43532998 0.27548134
 0.06987586 0.68181388 0.64756457 0.18136804 0.88466798 0.19942417
 0.0385368  0.4312381  0.55856187 0.55743958 0.65924505 0.68006215
 0.75321626 0.72458764 0.36890083 0.31565361 0.02460112 0.25667288
 0.0797472  0.65190499 0.69607954 0.65230246 0.03675547 0.34324064
 0.41411347 0.60966287 0.64602818 0.50818257 0.35868865 0.33985861
 0.45761426 0.19375051 0.2105351  0.97093743 0.2858085  0.33831274
 0.76760505 0.83376719 0.17297143 0.42487354 0.8292875  0.41171761
 0.30674687 0.97826775 0.31216793 0.41253795 0.94441928 0.65674818
 0.62890189 0.81527392 0.12294385 0.90380627 0.50398009 0.0365338
 0.80075509 0.5236378  0.50095102 0.21214549 0.9015869  0.50406753
 0.93513715 0.31088903 0.94677817 0.51617887 0.03564318 0.94079765
 0.68655681 0.91905829 0.26299852 0.01076258 0.43750913 0.80323554
 0.78465948 0.8171652  0.2857702  0.96391063

## 2. Generate Embeddings

In [59]:
# Cell 1: Generate Embeddings
# This code demonstrates how to generate embeddings for text using the OpenAI API.

def get_embedding(text):
    """
    Generate embeddings for text using OpenAI API
    """
    response = client.embeddings.create(
        model="text-embedding-3-small",  # A common and efficient embedding model
        input=text,
        encoding_format="float"  # Specify the format of the embedding vector
    )

    # The embedding vector is in the 'data' attribute of the response
    # It's a list of embedding objects, we take the first one's embedding
    embedding_vector = response.data[0].embedding
    return embedding_vector

# Example usage
text_to_embed = "This is a sample sentence for creating an embedding."

embedding_vector = get_embedding(text_to_embed)

# Print the embedding vector and its dimension
print("Text:", text_to_embed)
print("Embedding vector (first 10 elements):", embedding_vector[:10])
print("Dimension of the embedding vector:", len(embedding_vector))

Text: This is a sample sentence for creating an embedding.
Embedding vector (first 10 elements): [0.03280215, 0.011288634, 0.02859873, 0.0051769116, -0.003304069, -0.03293109, 0.03349842, -0.016040046, -0.024150325, 0.019882435]
Dimension of the embedding vector: 1536


## 3.Compare Embeddings

In [71]:
# Cell 2: Compare Embeddings

import numpy as np

def cosine_similarity(vec1, vec2):
    """
    Calculate cosine similarity between two vectors
    """
    v1 = np.array(vec1)
    v2 = np.array(vec2)
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# Example texts to compare
text1 = "How to train a cat?"
text2 = "How to drive a car?"

# Get embeddings for both texts
embedding1 = get_embedding(text1)
embedding2 = get_embedding(text2)

# Calculate similarity
similarity = cosine_similarity(embedding1, embedding2)

print(f"Text 1: {text1}")
print(f"Text 2: {text2}")
print(f"Similarity: {similarity:.4f}")

Text 1: How to train a cat?
Text 2: How to drive a car?
Similarity: 0.3499


# 3. Vector Database

## 1. Chroma DB

Solo dev / small app / on-prem or hybrid → start with Chroma (OSS, dead-simple API; can move to Chroma Cloud later). Chroma

### 1. Basic Setup

In [ ]:
# Cell 1: Setup
!pip install chromadb

import chromadb

chroma = chromadb.Client()
collection = chroma.get_or_create_collection("demo")

def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

print("✅ Ready!")

In [73]:
# Cell 2: Store Embeddings

texts = [
    "How do I cook pasta?",
    "The weather is sunny",
    "AI is amazing"
]

print("Storing embeddings...")

embeddings = [get_embedding(text) for text in texts]

collection.add(
    ids=["1", "2", "3"],
    documents=texts,
    embeddings=embeddings
)

print(f"✅ Stored {len(texts)} documents!")

Storing embeddings...
✅ Stored 3 documents!


In [74]:
# Cell 3: View Stored Data

# Show what's stored
# Explicitly include embeddings, documents, and metadatas
data = collection.get(
    include=['embeddings', 'documents', 'metadatas']
)

ids = data["ids"]
docs = data["documents"]
embs = data.get("embeddings")


print(f"Found {len(ids)} stored items:\n")

# If for some reason embeddings are missing, don't try to iterate over them
if embs is None or any(e is None for e in embs):
    for idx, doc in zip(ids, docs):
        print(f"ID: {idx}")
        print(f"Text: {doc}")
        print("(no embeddings returned)")
        print("-" * 40)
else:
    for idx, doc, emb in zip(ids, docs, embs):
        print(f"ID: {idx}")
        print(f"Text: {doc}")
        print(f"Embedding (first 5 dims): {emb[:5]}")
        print("-" * 40)

Found 3 stored items:

ID: 1
Text: How do I cook pasta?
Embedding (first 5 dims): [-0.06047147 -0.06067772 -0.01197261 -0.02462583  0.00011247]
----------------------------------------
ID: 2
Text: The weather is sunny
Embedding (first 5 dims): [ 0.00202606 -0.04834312  0.00734349 -0.00663184  0.02571756]
----------------------------------------
ID: 3
Text: AI is amazing
Embedding (first 5 dims): [-0.00992907 -0.00132574 -0.01422693  0.04163846  0.00857541]
----------------------------------------


#3. RAG

## 1. Set up Chroma + OpenAI API

In [75]:
import os
from getpass import getpass


# Create (or get) a collection to hold our chunks
collection = chroma.get_or_create_collection(name="rag_demo")

print("✅ Setup complete.")


✅ Setup complete.


## 2. Prepare and chunk a text

In [76]:
# A small sample text (replace with your own)
text = """
Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our



This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer. This improves accuracy and keeps responses up-to-date with our data.
"""

# Simple sentence-based chunking (groups of ~2–3 sentences)
import re

def chunk_text(t: str, max_sentences=1):
    sents = [s.strip() for s in re.split(r'(?<=[.!?])\s+', t.strip()) if s.strip()]
    chunks, buf = [], []
    for s in sents:
        buf.append(s)
        if len(buf) >= max_sentences:
            chunks.append(" ".join(buf))
            buf = []
    if buf:
        chunks.append(" ".join(buf))
    return chunks

chunks = chunk_text(text, max_sentences=2)
for i, c in enumerate(chunks):
    print(f"[{i}] {c}\n")

print(f"✅ Prepared {len(chunks)} chunks.")


[0] Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer.

[1] This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer.

[2] This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the ans

## 3. Turn chunks into embeddings

In [77]:
EMBED_MODEL = "text-embedding-3-small"  # 1536-dim, fast & inexpensive

def embed_texts(text_list):
    resp = client.embeddings.create(model=EMBED_MODEL, input=text_list)
    return [d.embedding for d in resp.data]

embeddings = embed_texts(chunks)

print(f"✅ Got {len(embeddings)} embeddings.")
print(f"   Each vector has dimension: {len(embeddings[0])}")


✅ Got 22 embeddings.
   Each vector has dimension: 1536


## 4.  Store in the vector DB (Chroma)

In [78]:
# Use string IDs "0", "1", ...
ids = [str(i) for i in range(len(chunks))]
metadatas = [{"source": "demo", "chunk_id": i} for i in range(len(chunks))]

collection.add(
    ids=ids,
    documents=chunks,
    embeddings=embeddings,
    metadatas=metadatas
)

# Peek at what we stored (without dumping full vectors)
data = collection.get(include=["documents", "metadatas", "embeddings"])  # add "embeddings" if you want them

print("✅ Stored items:")
for i, doc, emb, meta in zip(data["ids"], data["documents"],data['embeddings'], data["metadatas"]):
    print(f"- id={i}, meta={meta}\n  {doc}\n {emb}\n")


✅ Stored items:
- id=0, meta={'source': 'demo', 'chunk_id': 0}
  Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer.
 [-0.01603832  0.02678413  0.03635518 ... -0.00518432  0.02789704
 -0.00210372]

- id=1, meta={'chunk_id': 1, 'source': 'demo'}
  This improves accuracy and keeps responses up-to-date with our data.
 [-0.00189765  0.02580518  0.07134885 ... -0.00433957  0.00373362
  0.01648745]

- id=2, meta={'source': 'demo', 'chunk_id': 2}
  This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most simil

### Display the data

In [79]:
# Install Rich once in Colab
!pip -q install rich

data = collection.get(include=["documents", "metadatas", "embeddings"])

from rich.console import Console
from rich.table import Table
import json

def fmt_text(s, maxlen=120):
    s = s or ""
    return (s[:maxlen] + "…") if len(s) > maxlen else s

def fmt_emb(e, preview=8):
    if e is None:
        return "—"
    head = ", ".join(f"{x:.4f}" for x in e[:preview])
    tail = " …" if len(e) > preview else ""
    return f"[{head}{tail}]  (dim={len(e)})"

table = Table(show_header=True, header_style="bold magenta")
table.add_column("ID", style="cyan", no_wrap=True)
table.add_column("Text", style="white")
table.add_column("Embedding (preview)", style="green")
table.add_column("Metadata", style="yellow")

for i, doc, emb, meta in zip(data["ids"], data["documents"], data["embeddings"], data["metadatas"]):
    table.add_row(str(i), fmt_text(doc), fmt_emb(emb, 8), json.dumps(meta, ensure_ascii=False))

Console().print("Stored items:", style="bold")
Console().print(table)


Stored items:

┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃ Text                              ┃ Embedding (preview)                ┃ Metadata                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0  │ Retrieval-Augmented Generation    │ [-0.0160, 0.0268, 0.0364, 0.0264,  │ {"chunk_id": 0, "source": "demo"} │
│    │ (RAG) helps language models       │ -0.0171, 0.0024, -0.0029, 0.0235   │                                   │
│    │ answer questions by looking up    │ …]  (dim=1536)                     │                                   │
│    │ supporting information from a …   │                                    │                                   │
│ 1  │ This improves accuracy and keeps  │ [-0.0019, 0.0258, 0.0713, 0.0449,  │ {"chunk_id": 1, "source": "demo"} │
│    │ responses up-to-date with our     │ 0.0035, 0.0076, 0.0031, 0.0305 …]  │                                   │
│    │ data.                             │ (dim=1536)                         │                                   │
│ 2  │ This improves accuracy and keeps  │ [-0.0140, 0.0351, 0.0526, 0.0331,  │ {"chunk_id": 2, "source": "demo"} │
│    │ responses up-to-date with our     │ -0.0177, 0.0031, 0.0012, 0.0394 …] │                                   │
│    │ data.Retrieval-Augmented          │ (dim=1536)                         │                                   │
│    │ Generation (RAG) helps language … │                                    │                                   │
│ 3  │ This improves accuracy and keeps  │ [-0.0140, 0.0351, 0.0526, 0.0331,  │ {"source": "demo", "chunk_id": 3} │
│    │ responses up-to-date with our     │ -0.0177, 0.0031, 0.0012, 0.0394 …] │                                   │
│    │ data.Retrieval-Augmented          │ (dim=1536)                         │                                   │
│    │ Generation (RAG) helps language … │                                    │                                   │
│ 4  │ This improves accuracy and keeps  │ [-0.0140, 0.0351, 0.0526, 0.0331,  │ {"source": "demo", "chunk_id": 4} │
│    │ responses up-to-date with our     │ -0.0177, 0.0031, 0.0012, 0.0394 …] │                                   │
│    │ data.Retrieval-Augmented          │ (dim=1536)                         │                                   │
│    │ Generation (RAG) helps language … │                                    │                                   │
│ 5  │ This improves accuracy and keeps  │ [-0.0140, 0.0351, 0.0526, 0.0331,  │ {"source": "demo", "chunk_id": 5} │
│    │ responses up-to-date with our     │ -0.0177, 0.0031, 0.0012, 0.0394 …] │                                   │
│    │ data.Retrieval-Augmented          │ (dim=1536)                         │                                   │
│    │ Generation (RAG) helps language … │                                    │                                   │
│ 6  │ This improves accuracy and keeps  │ [-0.0140, 0.0351, 0.0526, 0.0331,  │ {"chunk_id": 6, "source": "demo"} │
│    │ responses up-to-date with our     │ -0.0177, 0.0031, 0.0012, 0.0394 …] │                                   │
│    │ data.Retrieval-Augmented          │ (dim=1536)                         │                                   │
│    │ Generation (RAG) helps language … │                                    │                                   │
│ 7  │ This improves accuracy and keeps  │ [-0.0140, 0.0351, 0.0526, 0.0331,  │ {"source": "demo", "chunk_id": 7} │
│    │ responses up-to-date with our     │ -0.0177, 0.0031, 0.0012, 0.0394 …] │                                   │
│    │ data.Retrieval-Augmented          │ (dim=1536)                         │                                   │
│    │ Generation (RAG) helps language … │                                    │                                   │
│ 8  │ This improves accuracy and keeps  │ [-0.0140, 0.0

In [ ]:
# Check what is stored in the Chroma collection
data = collection.get(
    include=['documents', 'metadatas'] # Include documents and metadatas
)

print("✅ Data currently stored in Chroma:")
if data and data['ids']:
    for i, (doc, meta) in enumerate(zip(data['documents'], data['metadatas'])):
        print(f"- ID: {data['ids'][i]}, Metadata: {meta}")
        print(f"  Document: {doc}\n")
else:
    print("No data found in the collection.")

print(f"📊 Total items in collection: {collection.count()}")

## 2. A simple vector search

In [81]:
# User query → embed → query Chroma
query = "What is RAG?"
q_emb = embed_texts([query])[0]

results = collection.query(
    query_embeddings=[q_emb],
    n_results=3,
    include=["distances", "documents", "metadatas"] # Removed 'ids' from include
)

print("🔎 Top matches:")
for i in range(len(results["ids"][0])):
    doc_id    = results["ids"][0][i]
    distance  = results["distances"][0][i]  # smaller is closer (cosine distance)
    doc_text  = results["documents"][0][i]
    meta      = results["metadatas"][0][i]
    print(f"\nRank {i+1} | id={doc_id} | distance={distance:.4f} | meta={meta}\n{doc_text}")

# From here you could pass the retrieved text to your LLM prompt to 'augment' generation.

🔎 Top matches:

Rank 1 | id=0 | distance=0.9758 | meta={'source': 'demo', 'chunk_id': 0}
Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer.

Rank 2 | id=7 | distance=1.0302 | meta={'chunk_id': 7, 'source': 'demo'}
This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language models answer questions by looking up
supporting information from a knowledge base. We chunk documents, embed those chunks,
store the vectors in a database, and at query time we retrieve the most similar chunks
to ground the answer.

Rank 3 | id=10 | distance=1.0302 | meta={'chunk_id': 10, 'source': 'demo'}
This improves accuracy and keeps responses up-to-date with our data.Retrieval-Augmented Generation (RAG) helps language mod